In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC   
from sklearn.model_selection import GridSearchCV                 
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix

In [2]:
# test = pd.read_csv('testing.csv', header=None, skiprows=1)

train = pd.read_csv('training.csv')
validation = pd.read_csv('validation.csv', header=None, skiprows=1)
X = pd.read_csv('training.csv')


In [3]:
# replace labels with ints
train.replace('blues', 1, inplace=True)
train.replace('classical', 2, inplace=True)
train.replace('country', 3, inplace=True)
train.replace('disco', 4, inplace=True)
train.replace('hiphop', 5, inplace=True)
train.replace('jazz', 6, inplace=True)
train.replace('metal', 7, inplace=True)
train.replace('pop', 8, inplace=True)
train.replace('reggae', 9, inplace=True)
train.replace('rock', 10, inplace=True)
train.drop(['song_name'], axis=1, inplace=True)
train.head()

,mean_spectral_rolloff,std_spectral_rolloff,mean_spectral_centroid,std_spectral_centroid,mean_spectral_bandwidth,std_spectral_bandwidth,mean_spectral_flatness,std_spectral_flatness,mean_spectral_contrast,std_spectral_contrast,...,std_mfcc_15,mean_mfcc_16,std_mfcc_16,mean_mfcc_17,std_mfcc_17,mean_mfcc_18,std_mfcc_18,mean_mfcc_19,std_mfcc_19,label
0,3805.839606,949.476395,1784.165850,360.241675,2002.449060,293.057608,0.004498,0.005513,20.526699,8.760242,...,7.240229,-1.690215,6.043515,-0.408979,6.449582,-2.303523,7.420440,1.221290,6.850987,1
1,3042.260232,885.457204,1552.811865,395.559911,1747.702312,276.141616,0.002631,0.003345,22.197517,8.725292,...,6.371716,-7.729093,6.902132,-1.816407,7.237551,-3.439720,6.829324,-2.231259,5.529287,1
2,2184.745799,1221.963322,1070.106615,429.366909,1596.412872,407.972419,0.000954,0.001631,21.426268,7.961446,...,6.665415,-3.319596,7.085667,0.636965,6.108939,-0.619121,6.104075,-3.407449,5.652375,1
3,3579.757627,1254.184130,1835.004266,586.003361,1748.172116,297.397392,0.003238,0.004356,21.466338,7.936384,...,9.278968,-5.454034,8.675812,-0.916874,7.322153,-4.404826,7.931634,-11.703234,7.429345,1
4,3481.517592,1809.541367,1831.993940,1015.126775,1729.653287,449.344532,0.003061,0.006482,22.545831,8.176806,...,8.517583,-1.838263,8.288669,-2.783799,6.515171,-3.047909,6.309420,-8.109991,6.805219,1


In [4]:
# check to see if all labels are ints
train['label'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [5]:
X.drop(['song_name'], axis=1, inplace=True)
X.drop(['label'], axis=1, inplace=True)
X.head()

,mean_spectral_rolloff,std_spectral_rolloff,mean_spectral_centroid,std_spectral_centroid,mean_spectral_bandwidth,std_spectral_bandwidth,mean_spectral_flatness,std_spectral_flatness,mean_spectral_contrast,std_spectral_contrast,...,mean_mfcc_15,std_mfcc_15,mean_mfcc_16,std_mfcc_16,mean_mfcc_17,std_mfcc_17,mean_mfcc_18,std_mfcc_18,mean_mfcc_19,std_mfcc_19
0,3805.839606,949.476395,1784.165850,360.241675,2002.449060,293.057608,0.004498,0.005513,20.526699,8.760242,...,0.752740,7.240229,-1.690215,6.043515,-0.408979,6.449582,-2.303523,7.420440,1.221290,6.850987
1,3042.260232,885.457204,1552.811865,395.559911,1747.702312,276.141616,0.002631,0.003345,22.197517,8.725292,...,2.451690,6.371716,-7.729093,6.902132,-1.816407,7.237551,-3.439720,6.829324,-2.231259,5.529287
2,2184.745799,1221.963322,1070.106615,429.366909,1596.412872,407.972419,0.000954,0.001631,21.426268,7.961446,...,0.780873,6.665415,-3.319596,7.085667,0.636965,6.108939,-0.619121,6.104075,-3.407449,5.652375
3,3579.757627,1254.184130,1835.004266,586.003361,1748.172116,297.397392,0.003238,0.004356,21.466338,7.936384,...,-4.520575,9.278968,-5.454034,8.675812,-0.916874,7.322153,-4.404826,7.931634,-11.703234,7.429345
4,3481.517592,1809.541367,1831.993940,1015.126775,1729.653287,449.344532,0.003061,0.006482,22.545831,8.176806,...,-5.576590,8.517583,-1.838263,8.288669,-2.783799,6.515171,-3.047909,6.309420,-8.109991,6.805219


In [6]:
y = train['label'].copy()
y.head()

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [7]:
# Normalization of Data
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X.iloc[:, :-1], dtype = float))


In [8]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [9]:
y_train.head()

575    8
108    2
46     1
132    2
220    3
Name: label, dtype: int64

In [10]:
#y_test.head()
y_test.shape

(186,)

In [11]:
# C: how much you want to avoid misclassifying each training example.
# # Gamma: how far the influence of a single training example reaches

parameters = {'kernel':('linear', 'rbf'),
              'C':[1, 10, 20]
             }

cls=SVC()
model = GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       n_jobs=1,
       param_grid=parameters,
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)
model.fit(X_train,y_train,groups=None)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5, error_score='raise', estimator=SVC(gamma=0.001), n_jobs=1,
             param_grid={'C': [1, 10, 20], 'kernel': ('linear', 'rbf')},
             return_train_score='warn', verbose=1)

In [12]:
# Testing 

test_pred = model.predict(X_test)

np.savetxt("accuracy_solution.csv", 
           np.dstack((np.arange(1, test_pred.size+1),test_pred))[0],
           delimiter=',', comments="", fmt='%i,%i',
           header="Sample_id,Sample_label")

In [13]:
print("Accuracy Score: ", accuracy_score(y_test, test_pred))

conf_matrix = confusion_matrix(y_test, test_pred)
print('\033[1m' + "\n Table 1: Confusion Matrix of Training Data \n\n" + '\033[0m', conf_matrix)

Accuracy Score:  0.7688172043010753

 Table 1: Confusion Matrix of Training Data 

 [[15  0  2  0  0  0  1  0  0  3]
 [ 0 17  1  0  0  1  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  2]
 [ 0  0  0 14  1  1  1  0  0  2]
 [ 0  0  0  2 12  0  0  0  3  0]
 [ 2  0  0  0  0 16  0  0  0  1]
 [ 0  0  0  2  0  0 17  0  0  0]
 [ 0  0  2  1  1  0  0 13  0  1]
 [ 2  0  0  0  1  0  0  2 12  1]
 [ 2  0  2  2  0  1  0  0  0 11]]
